In [36]:
import urllib
import requests
import os
import subprocess
import shutil

# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [37]:
def get_app(app_url):
    app_url += '/download?from=details'
    download_page = requests.get(app_url)
    soup = BeautifulSoup(download_page.content)
    apk_url = soup.select('#iframe_download')[0].attrs['src']
    return requests.get(apk_url).content

In [38]:
# k = get_app('https://apkpure.com/wallpaper-for-iphone-x-8-8/com.wallpapersforiphoneX.themeapplock88plus')

In [39]:
def get_data(**config):
    data_dir = config['data_dir']
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        
    apps_dir = os.path.join(data_dir, 'apps')
    if not os.path.exists(apps_dir):
        os.mkdir(apps_dir)
    
    for url in config['urls']:
        app_slug, app_dev = url.split('/')[-2:]
        app_slug = urllib.parse.unquote(app_slug)  # include or not
        dev_dir = os.path.join(apps_dir, app_dev)
        if not os.path.exists(dev_dir):
            os.mkdir(dev_dir)
        
        apk = get_app(url)
        
        apk_fn = os.path.join(dev_dir, app_slug + '.apk')
        apk_fp = open(apk_fn, 'wb')
        apk_fp.write(apk)
        print(apk_fn)
        
        code_dir = os.path.join(dev_dir, app_slug)
        if os.path.exists(code_dir):
            shutil.rmtree(code_dir)
            
        command = subprocess.run([
            'apktool', 'd',  # decode
            '-r',            # no resources
            apk_fn,          # apk filename
            '-o', code_dir   # decompiled out path
        ], capture_output=True)
        print(command.stdout.decode())

In [40]:
get_data(**cfg)

../data/apps/com.wallpapersforiphoneX.themeapplock88plus/wallpaper-for-iphone-x-8-8.apk
I: Using Apktool 2.4.1 on wallpaper-for-iphone-x-8-8.apk
I: Copying raw resources...
I: Baksmaling classes.dex...
I: Copying assets and libs...
I: Copying unknown files...
I: Copying original files...

../data/apps/com.instagram.android/instagram.apk
I: Using Apktool 2.4.1 on instagram.apk
I: Copying raw resources...
I: Baksmaling classes.dex...
I: Baksmaling classes2.dex...
I: Baksmaling classes3.dex...
I: Baksmaling classes4.dex...
I: Copying assets and libs...
I: Copying unknown files...
I: Copying original files...



In [17]:
cfg = {
    'data_dir': '../data',
    'urls': [
        'https://apkpure.com/wallpaper-for-iphone-x-8-8/com.wallpapersforiphoneX.themeapplock88plus',
        'https://apkpure.com/instagram/com.instagram.android'
    ]
}